In [1]:
!pip install keras
!pip install tensorflow
import tensorflow as tf
import numpy as np

In [2]:
from tensorflow.python.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GRU, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [3]:
mark_start = 'ssss ' #başlangıç değeri yani modelimize burası başlangıç noktası diyoruz
mark_end = ' eeee' #bitiş değeri; modele nerde durması gerektiğini belirtiyoruz.

In [4]:
data_src = [] #veri listelerimiz source kısmına ingilizce yapılar konulacak
data_dest = [] #türkçe veriler konulacak.

In [5]:
for line in open('tur.txt', encoding='UTF-8'):
    en_text, tr_text = line.rstrip().split('\t')

    tr_text = mark_start + tr_text + mark_end #başlangıç ve bitiş işaretlerimiz ekliyoruz "ssss abc eeee"

    data_src.append(en_text)
    data_dest.append(tr_text)

In [6]:
data_src[500]

'I hope so.'

In [7]:
data_dest[500] #

'ssss Öyle olduğunu umuyorum. eeee'

In [8]:
data_src[50000]

'Do you know my name?'

In [9]:
data_dest[50000]

'ssss Adımı biliyor musun? eeee'

In [10]:
len(data_src)

473035

In [11]:
##TOKENIZER sınıfından kalıtım alarak bazı işlevler ekleyeceğiz##

class TokenizerWrap(Tokenizer):
    def __init__(self, texts, padding, reverse=False, num_words=None): #init fonksiyonu classımızı her çağırdığımızda çalışacak yapıları belirtir
        Tokenizer.__init__(self, num_words=num_words)

        self.fit_on_texts(texts)
        self.index_to_word = dict(zip(self.word_index.values(), self.word_index.keys())) #kelimelere index vererek bir dictionary oluşturuyoruz.
        self.tokens = self.texts_to_sequences(texts)

        if reverse:
            self.tokens = [list(reversed(x)) for x in self.tokens]
            truncating = 'pre'          #pre ve post olarak ikiye ayrılıyor
        else:                           #dizileri ne tarafa ekleyeceğimizi nereden keseceğimizi belirtir.
            truncating = 'post'


        #max uzunluğu belirliyoruz
        self.num_tokens = [len(x) for x in self.tokens]
        self.max_tokens = np.mean(self.num_tokens) + 2 * np.std(self.num_tokens)
        self.max_tokens = int(self.max_tokens)

        #padding(ekleme ve silme) işlemi
        self.tokens_padded = pad_sequences(self.tokens,
                                           maxlen=self.max_tokens,
                                           padding=padding,
                                           truncating=truncating)

    def token_to_word(self, token): #tokenleri metne çevirir.
        word = ' ' if token == 0 else self.index_to_word[token]
        return word

    def tokens_to_string(self, tokens): #token dizisini (sayılar) tekrar metne çevirir.
        words = [self.index_to_word[token] for token in tokens if token != 0]
        text = ' '.join(words)
        return text

    def text_to_tokens(self, text, padding, reverse=False): #metni token şeklinde verir
        tokens = self.texts_to_sequences([text])
        tokens = np.array(tokens)

        if reverse:
            tokens = np.flip(tokens, axis=1)
            truncating = 'pre'
        else:
            truncating = 'post'

        tokens = pad_sequences(tokens,
                               maxlen=self.max_tokens,
                               padding=padding,
                               truncating=truncating)

        return tokens

In [12]:
tokenizer_src = TokenizerWrap(texts=data_src,  #oluşturduğumuz "TokenizerWrap" sınıfını kullanıyoruz
                              padding='pre',   #başına sıfırlar eklenerek doldurulacak.
                              reverse=True,    #cümle sırası ters çevrilecek
                              num_words=None)

In [13]:
tokenizer_dest = TokenizerWrap(texts=data_dest,
                              padding='post',  #başına sıfırlar eklenerek doldurulacak.
                              reverse=False,   #cümle ters çevirlmeyecek
                              num_words=None)

In [14]:
# token dizilerine erişim #
tokens_src = tokenizer_src.tokens_padded
tokens_dest = tokenizer_dest.tokens_padded
print(tokens_src.shape) #tokenlenmiş source listemizin boyutunu yazdırır
print(tokens_dest.shape) #tokenlenmiş destination(türkçe) listemizin boyutu

(473035, 11)
(473035, 10)


In [15]:
tokens_dest[50000]

array([   1, 2233,  108,   48,    2,    0,    0,    0,    0,    0],
      dtype=int32)

In [16]:
tokenizer_dest.tokens_to_string(tokens_dest[50000])

'ssss adımı biliyor musun eeee'

In [17]:
tokens_src[50000]

array([  0,   0,   0,   0,   0,   0, 288,  24,  33,   5,   9], dtype=int32)

In [18]:
tokenizer_src.tokens_to_string(tokens_src[200000]) #source lsitemizdeki tokenleştirilmiş cümleleri tekrar metne çevirerek ters çevrilmiş halini verir

'missing anything see you can'

In [19]:
data_src[50000]

'Do you know my name?'

In [20]:
token_start = tokenizer_dest.word_index[mark_start.strip()]
token_start # "ssss " başlangıç markımızın indexi

1

In [21]:
token_end = tokenizer_dest.word_index[mark_end.strip()]
token_end # " eeee" bitiş markımızın indexi

2

In [22]:
## ENCODER VE DECODER ICIN GIRIS VE CIKIS DEGERLERINI HAZIRLAMA ##

encoder_input_data = tokens_src
decoder_input_data = tokens_dest[:, :-1] #ürkçe cümlelerin başlangıçtan bir önceki hali
decoder_output_data = tokens_dest[:, 1:] #Türkçe cümlelerin bir sonraki hali

In [23]:
encoder_input_data[50000]

array([  0,   0,   0,   0,   0,   0, 288,  24,  33,   5,   9], dtype=int32)

In [24]:
decoder_input_data[50000]

array([   1, 2233,  108,   48,    2,    0,    0,    0,    0], dtype=int32)

In [25]:
decoder_output_data[50000]

array([2233,  108,   48,    2,    0,    0,    0,    0,    0], dtype=int32)

In [26]:
tokenizer_dest.tokens_to_string(decoder_input_data[50000])

'ssss adımı biliyor musun eeee'

In [27]:
tokenizer_dest.tokens_to_string(decoder_output_data[50000])

'adımı biliyor musun eeee'

In [28]:
num_encoder_words = len(tokenizer_src.word_index)
num_decoder_words = len(tokenizer_dest.word_index)

In [29]:
num_encoder_words

21315

In [30]:
num_decoder_words

94058

In [31]:
embedding_size = 100 #her kelime 100 boyutlu matrisle ifade edilecek

In [32]:
word2vec = {}
with open('glove.6B.100d.txt', encoding='UTF-8') as f: #glove ile eğitilmil 6 milyar 100 boyutlu ingilizce kelime vektörlerini içerir
    for line in f:                                     #her satır: "word 0.1 0.25 -0.3 ..." şeklinde
        values = line.split()
        word = values[0]
        vec = np.asarray(values[1:], dtype='float32')
        word2vec[word] = vec

#word2vec sözlüğünü oluşturuyoruz exp: "cat": [0.1, 0.25, ...]

In [33]:
# embedding matrixini oluşturma #
embedding_matrix = np.random.uniform(-1, 1, (num_encoder_words, embedding_size))
for word, i in tokenizer_src.word_index.items(): #ingilizce kelimeler ve onların indeksleri
    if i < num_encoder_words:
        embedding_vector = word2vec.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector #o kelimeye karşılık gelen indeks pozisyonuna matrix verilir.

In [34]:
embedding_matrix.shape #oluşturduğumuz embedding matrisinin boyutu

(21315, 100)

In [35]:
encoder_input = Input(shape=(None,), name='encoder_input') #encoder yani ingilizce tarafta çalışan kısım
                                                           #shape none olması cümle uzunluğu değişken olabilir

In [36]:
#ingilizce kelimeler embedding vektörlerine çevirme

encoder_embedding = Embedding(input_dim=num_encoder_words,
                              output_dim=embedding_size,
                              weights=[embedding_matrix], #daha önce oluşturduğumuz gloVe matrisini kullanır
                              trainable=True,             #eğitim sırasında matrisler güncellenebilir
                              name='encoder_embedding')

In [37]:
#encoder (ingilizce) tarafı için 3 katlı gru
state_size = 256
encoder_gru1 = GRU(state_size, name='encoder_gru1', return_sequences=True)
encoder_gru2 = GRU(state_size, name='encoder_gru2', return_sequences=True)  #true olunca tüm adımların çıktısını döner
encoder_gru3 = GRU(state_size, name='encoder_gru3', return_sequences=False) #false olunca sadece son adımın çıktısını döner

In [38]:
# encoder katmanlarını bağlama #
def connect_encoder(encoder_input):
    net = encoder_embedding(encoder_input)
    net = encoder_gru1(net)
    net = encoder_gru2(net)
    net = encoder_gru3(net)
    return net  # encoder_output
 #encoder_output : Sabit boyutta bir vektör, Türkçeye çevrilecek İngilizce cümlenin anlam temsili.

#tüm encoder katmanlarını sırayla birbirine bağlar.

In [39]:
encoder_output = connect_encoder(encoder_input)  #encoderı çalıştırma

In [40]:
decoder_initial_state = Input(shape=(state_size,), name='decoder_initial_state')  #decoder yani türkçe tarafta çalışan kısım

In [41]:
decoder_input = Input(shape=(None,), name='decoder_input')

In [42]:
# decoder (türkçe) kısım için embedding matrisleri oluşturma #
decoder_embedding = Embedding(input_dim=num_decoder_words,
                              output_dim=embedding_size,
                              name='decoder_embedding')

In [43]:
# 3 katlı gru eğitimi
decoder_gru1 = GRU(state_size, name='decoder_gru1', return_sequences=True)
decoder_gru2 = GRU(state_size, name='decoder_gru2', return_sequences=True)
decoder_gru3 = GRU(state_size, name='decoder_gru3', return_sequences=True) #hepisinin true olmasının sebebi her kelime üretiiminde çıktı üretmesi gerektiğimnden

In [44]:
# dense yani çıktı katmanı oluşturma #
decoder_dense = Dense(num_decoder_words,
                      activation='linear',   #türkçedeki toplam kelime sayısı kadar nöron içerir
                      name='decoder_output') #Sonrasında softmax uygulanacak, o yüzden burası lineer bırakılır.

In [45]:
# decoder için gru ları bağlama
def connect_decoder(decoder_input, initial_state):
    net = decoder_embedding(decoder_input)
    net = decoder_gru1(net, initial_state=initial_state)  # İlk katmana encoder’dan gelen state
    net = decoder_gru2(net)   # Sonraki katmanlarda initial_state yok
    net = decoder_gru3(net)
    decoder_output = decoder_dense(net)
    return decoder_output

In [48]:
# EĞİTİM MODELİ #
# Inputları tanımlama
decoder_input = Input(shape=(None,), name='decoder_input')

# fonksiyonları kullanarak ağları bağla
encoder_output = connect_encoder(encoder_input)
decoder_output = connect_decoder(decoder_input, initial_state=encoder_output)

# modeli tanımla
model_train = Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])
                                                                                     #encoder input ingilizce cümle,
                                                                                     #decoder input türkçe cümlenin başlangıç kısmı ssss
                                                                                     #decoder output türkçeye çevrilen cümlemiz

In [49]:
model_encoder = Model(inputs=[encoder_input], outputs=[encoder_output]) #Sadece ingilizce tarafı, tahmin aşamasında önce bu çalıştırılır.

In [50]:
decoder_output = connect_decoder(decoder_input, initial_state=decoder_initial_state) #decoder input : şu ana kadar üretilen Türkçe kelimeler.

model_decoder = Model(inputs=[decoder_input, decoder_initial_state], outputs=[decoder_output]) #decoder_initial_state: encoder çıktısı)

In [51]:
# LOSS KAYIP FONKSIYONU #
def sparse_cross_entropy(y_true, y_pred):
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred) # y_true: gerçek türkçe kelime tokenleri.
                                                                                        # y_pred: modelin tahmin ettiği skorlar.
    loss_mean = tf.reduce_mean(loss)
    return loss_mean #ortalama kayıp değeri

In [52]:
optimizer = RMSprop(learning_rate=1e-3) # öğrenme oranı: 1e-3 : 0.001

In [53]:
decoder_target = Input(shape=(None,), dtype='int32', name='decoder_target')


In [54]:
model_train.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [55]:
path_checkpoint = 'checkpoint.weights.h5'  #eğitim sırasında modelin ağırlıkları bu dosyaya kaydedilir.
checkpoint = ModelCheckpoint(filepath=path_checkpoint, save_weights_only=True) #save_weights_only=True: Sadece weights saklanır model yapısı değil.

In [56]:
try:
    model_train.load_weights(path_checkpoint)
except Exception as error:
    print('Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.')
    print(error)
#önceden eğitilmiş bir model varsa onu yükler
#hata varsa(örneğin dosya yoksa) sıfırdan başlar.

Checkpoint yüklenirken hata oluştu. Eğitime sıfırdan başlanıyor.
[Errno 2] Unable to synchronously open file (unable to open file: name = 'checkpoint.weights.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)


In [57]:
x_data = {'encoder_input': encoder_input_data, 'decoder_input': decoder_input_data}
y_data = {'decoder_output': decoder_output_data}
#encoder_input_data: kaynak yani ingilizce cümleler.
#decoder_input_data: Türkçe çevirinin giriş kısmı (başlangıç ssss dahil ama son kelime yok)
#decoder_output_data: Türkçe çevirinin hedef kısmı (ilk kelime yok ama eeee dahil).

In [58]:
model_train.fit(x=x_data,
                y=y_data,
                batch_size=256,
                epochs=0, #sadece modelin yapısını çalıştırır, bunu 10–20 yaparsak gerçek eğitim başlar.
                callbacks=[checkpoint]) #eğitim sırasında her epoch sonunda modeli kaydeder.

In [59]:
def translate(input_text, true_output_text=None):
    # Input text'i tokenize et
    input_tokens = tokenizer_src.text_to_tokens(
        text=input_text,
        reverse=True,
        padding='pre'
    )

    # Eğer input_tokens tek bir cümleyse, batch haline getir
    if len(input_tokens.shape) == 1:
        input_tokens = np.expand_dims(input_tokens, axis=0)

    max_tokens = tokenizer_dest.max_tokens
    decoder_input_data = np.zeros((1, max_tokens), dtype=int)  # decoder girişi sıfırlarla başlatıldı.

    # Encoder'dan başlangıç durumu alınır
    initial_state = model_encoder.predict(input_tokens, verbose=0)
    print(f"Initial state shape: {initial_state.shape}")
    state_size = initial_state.shape[1]  # state_size'i encoder çıkış boyutuna göre ayarlıyoruz
    initial_state = initial_state.reshape(1, state_size)  # Hedeflenmiş şekle dönüştürülür.

    token_int = token_start  # Başlangıç token'ı
    output_text = ''  # Çıktıyı burada biriktireceğiz
    count_tokens = 0  # Token sayısı

    while token_int != token_end and count_tokens < max_tokens:
        decoder_input_data[0, count_tokens] = token_int  # Şu anki token'ı decoder girişine ekle

        # Decoder'a veri gönder
        x_data = {
            'decoder_input': decoder_input_data[:, :count_tokens + 1],  # Sadece gerekli token'ları ver
            'decoder_initial_state': initial_state  # Encoder'dan gelen initial state
        }

        # Decoder modelinden çıktı alınır
        decoder_output = model_decoder.predict(x_data, verbose=0)

        # Çıktıdan token alınır
        token_onehot = decoder_output[0, count_tokens, :]
        token_int = np.argmax(token_onehot)  # En yüksek olasılığa sahip token alınır

        # Token'ı kelimeye dönüştür
        sampled_word = tokenizer_dest.token_to_word(token_int)
        output_text += ' ' + sampled_word  # Kelimeyi çıktıya ekle

        count_tokens += 1  # Token sayısını artır

    # Sonuçları yazdır
    print('Input text:')
    print(input_text)
    print()

    print('Translated text:')
    print(output_text.strip())
    print()

    if true_output_text is not None:
        print('True output text:')
        print(true_output_text)
        print()


In [63]:
translate(input_text=data_src[400000], true_output_text=data_dest[400000])

Initial state shape: (1, 128)


ValueError: Input 1 of layer "functional_3" is incompatible with the layer: expected shape=(None, 256), found shape=(1, 128)

In [61]:
translate(input_text='Which road leads to the airport?')

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['encoder_input']
Received: inputs=Tensor(shape=(1, 11))
  warnings.warn(msg)


InvalidArgumentError: Graph execution error:

Detected at node functional_2_1/embedding_1/GatherV2 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.1